## CloudFormation script to automate cross-account resource registration on central Model Registry

* Please reach out to Yuyao Zhang ozhang@amazon.com or Melanie Li mmelli@amazon.com for any issue or questions

In [ ]:
!pip install boto3 sagemaker

The role for CloudFormation requires `SageMakerFullAccess` and following additional permissions:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "iam:GetRole",
                "events:DescribeRule",
                "events:PutRule",
                "iam:DeletePolicy",
                "iam:CreateRole",
                "iam:DeleteRole",
                "iam:AttachRolePolicy",
                "iam:PutRolePolicy",
                "iam:CreatePolicy",
                "events:PutTargets",
                "events:DeleteRule",
                "iam:PassRole",
                "iam:DetachRolePolicy",
                "iam:DeleteRolePolicy",
                "events:RemoveTargets",
                "iam:GetRolePolicy",
                "iam:DeletePolicyVersion"
            ],
            "Resource": "*"
        }
    ]
}
```

In [19]:
%%writefile prod-account-template.yaml
---
AWSTemplateFormatVersion: '2010-09-09'

Description: >-
  Trigger SageMaker Lineage Creation in Root account.
Parameters:
  RootAccountNumber:
    Type: String
    Description: Root acount where events should be sent to.
        
Resources:

  #### SECTION: Triggering the Lambda Automatically
  # Until SageMaker directly supports EventBridge events on 'apps' (as notebook instance statuses already are), we can
  # work around this by triggering the function via a CloudTrail. 

  CloudWatchEventRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
          - Effect: Allow
            Principal:
              Service:
                - events.amazonaws.com
            Action: sts:AssumeRole
      Path: /
      Policies:
        - PolicyName: AllowPutEvents
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action: 'events:PutEvents'
                Resource: !Sub 'arn:aws:events:${AWS::Region}:${RootAccountNumber}:event-bus/default'

  # Finally, the rule controls how CloudTrail-logged events trigger the Lambda
  CloudWatchEventRule:
    Type: 'AWS::Events::Rule'
    Properties:
      EventPattern:
        source:
          - aws.sagemaker
        detail-type:
          - 'AWS API Call via CloudTrail'
        detail:
          eventSource:
            - sagemaker.amazonaws.com
          eventName:
            - CreateModel
          requestParameters:
            containers:
              modelPackageName:
              - exists: true
      Targets:
        - Arn: !Sub 'arn:aws:events:${AWS::Region}:${RootAccountNumber}:event-bus/default'
          Id: api-based-createmodel-lineage
          RoleArn: !GetAtt CloudWatchEventRole.Arn

Overwriting prod-account-template.yaml


In [21]:
root_account_id = 607162686141

In [23]:
!aws cloudformation create-stack \
  --stack-name SageMakerLineageExtention\
  --template-body file://./prod-account-template.yaml \
  --parameters ParameterKey=RootAccountNumber,ParameterValue=$root_account_id \
  --role-arn arn:aws:iam::043207074741:role/cfntest \
  --capabilities CAPABILITY_IAM  

{
    "StackId": "arn:aws:cloudformation:ap-southeast-2:043207074741:stack/SageMakerLineageExtention/5af09230-dc2c-11ec-9ded-02a200717f74"
}


In [26]:
import sagemaker

sess = sagemaker.session.Session()

modelPkg = sagemaker.model.ModelPackage(name='cross-account-test-00',
                     model_package_arn="arn:aws:sagemaker:ap-southeast-2:607162686141:model-package/woodside-new-p-6dvnevzoh2mt/2", 
                     role="arn:aws:iam::043207074741:role/service-role/AmazonSageMaker-ExecutionRole-20220522T213009",
                     sagemaker_session=sess)

# Create SageMaker Model now
modelPkg._create_sagemaker_model()